# API development for download of Sentinel-2 and Landsat-8 data
### User defined mosaicing of harmonised products
_Robin Kohrs & Konstantin Schellenberg, February 2020, GEO 419_  
_Supervisor: John Truckenbrodt¹, Martin Habermeyer²_ <br>
<sub><sub>
¹ Friedrich-Schiller-University Jena, chair of remote sensing <br>
² Deutsches Luft- und Raumfahrtszentrum, earth observation center
</sub></sub>

## Tutorial
This is the extention for nasa_hls. We developed following new functions:

In order to make the program be more flexible, the goal of this extension is to
1. Download tiles by setting spatial (user defined verctor geometry) and temporal inqueries to the HLS server,
2. Mosaic the HDF4 formatted product per band and acquisition date.

As part of the module report will want to add the following post procession tasks: 
3. Calculate spectral indexes from the downloaded products.
4. Plot the results inline with `ipyleafet` and `folium` on a Open Street Map WMS service.

Tasks 1 and 2 are meant to work in accordance with the `nasa_hls` package and aims to lead to a pull request to the repository; 3 and 4 shall give an overview of to use the new utilities and the HLS product.

User guide to the HLS product:
https://hls.gsfc.nasa.gov/wp-content/uploads/2019/01/HLS.v1.4.UserGuide_draft_ver3.1.pdf

## Download HSL files with user input
First, the module nasa_hls is loaded.

In [6]:
# change working directory 
import os
os.chdir("/home/robin/python_projects/nasa_hls")

In [14]:
import warnings
warnings.simplefilter("ignore")

In [8]:
%matplotlib inline

import nasa_hls
import sys
import pandas as pd
import os

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# and for later processing in the notebook

For testing purpose, try downloading the kml file:

In [9]:
nasa_hls.download_kml()

UTM tiles already successfully downloaded to:
 /home/robin/.nasa_hls/.auxdata/utm.kml 



'/home/robin/.nasa_hls/.auxdata/utm.kml'

## Make list with GeoDataFrames

- This will serve as a conformation for the user
- It's a list of lists. Each inner list beeing the GeoDataFrames for the specified dates

In [15]:
ds = nasa_hls.make_tiles_dataset(shape="/home/robin/python_projects/data/nasa_hls/test_shape/dummy_region.dbf",
                                products=["S30"],
                                start_date="2018-12-25")


starting date: 2018-12-25
 
UTM tiles already successfully downloaded to:
 /home/robin/.nasa_hls/.auxdata/utm.kml 



  0%|          | 0/4 [00:00<?, ?it/s]


getting available datasets . . .


100%|██████████| 4/4 [00:07<00:00,  1.97s/it]


In [35]:
# print the list of lists
print(ds, "\n\n")
print("The query starting at date '2018-12-25', resulted in {len} hls-prodcuts".format(len = len(ds)))


[    product   tile       date                                                url
65      S30  34JDN 2018-12-27  https://hls.gsfc.nasa.gov/data/v1.4/S30/2018/3...
200     S30  34JEN 2018-12-27  https://hls.gsfc.nasa.gov/data/v1.4/S30/2018/3...
268     S30  34JDP 2018-12-27  https://hls.gsfc.nasa.gov/data/v1.4/S30/2018/3...
403     S30  34JEP 2018-12-27  https://hls.gsfc.nasa.gov/data/v1.4/S30/2018/3...,     product   tile       date                                                url
201     S30  34JEN 2018-12-29  https://hls.gsfc.nasa.gov/data/v1.4/S30/2018/3...
404     S30  34JEP 2018-12-29  https://hls.gsfc.nasa.gov/data/v1.4/S30/2018/3...] 


The query starting at date '2018-12-25', resulted in 2 hls-scenes


As one can see in the list, there are 4 scenes for the 27th of december and two products for the 29th.

Taking this list, the data sources can be downloaded via `download_tiles`. 

This function call `download_batch` and other methods internally in order to parse the right URLs for download.

The path for the parameter `dir` needs to be adapted as this is user-specific.

In [39]:
nasa_hls.download_tiles(dstdir=os.path.join("/home/robin/python_projects/data/nasa_hls/hdf_tiles"),
                        dataframes=ds)

100%|██████████| 2/2 [01:28<00:00, 44.44s/it]


***
## Mosaicing tiles
To simplify the handling of the downloaded files ... we mosaic the data according to the 
`make_mosaic.py`

In [ ]:
path_in = os.path.join(os.path.expanduser('~'), 'Dokumente', 'nasa_hls', 'data', 'hdf' + os.sep)
path_out =  os.path.join(os.path.expanduser('~'), 'Dokumente', 'nasa_hls', 'data', 'tifs' + os.sep)

In [ ]:
make_mosaic_tif()

The output extent can now be clipped to the actual outline of the input geometry.

## Calculate Indexes